In [1]:
#Import libraries and data

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

In [3]:
fake_news=pd.read_csv("Fake.csv")
true_news=pd.read_csv("True.csv")

In [7]:
true_news['label'] = 0

In [8]:
true_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [9]:
fake_news['label'] = 1

In [10]:
fake_news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [11]:
dataset1 = true_news[['text','label']]
dataset2 = fake_news[['text','label']]

In [13]:
dataset = pd.concat([dataset1 , dataset2])

In [14]:
dataset.shape

(44898, 2)

In [22]:
dataset.isnull().sum() # no null values

text     0
label    0
dtype: int64

In [23]:
dataset['label'].value_counts()

1    23481
0    21417
Name: label, dtype: int64

In [24]:
dataset1.shape # true news

(21417, 2)

In [25]:
dataset2.shape # fake news

(23481, 2)

### Shuffle or Resample

In [27]:
dataset = dataset.sample(frac = 1)

In [29]:
dataset.head(20)

,text,label
13517,Trump will announce his VP pick tomorrow. Rumo...,1
8847,WASHINGTON (Reuters) - Republican presidential...,0
338,Unless you ve been living under a rock since J...,1
13817,A Texas Imam agrees with Republican presidenti...,1
22776,Just look around the world and throughout rece...,1
16466,ISLAMABAD (Reuters) - A Pakistani court issued...,0
12310,Just imagine how many brilliant young American...,1
12886,JERUSALEM (Reuters) - The Israeli military sai...,0
13476,WASHINGTON/LONDON (Reuters) - President Donald...,0
6586,MOSCOW (Reuters) - Russia’s Vladimir Putin sai...,0


In [35]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
ps = WordNetLemmatizer()
stopwords = stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pranjalshukla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
def cleaning_data(row):
    
    # convert text to into lower case
    row = row.lower() 
    
    # this line of code only take words from text and remove number and special character using RegX
    row = re.sub('[^a-zA-Z]' , ' ' , row)
    
    # split the data and make token.
    token = row.split() 
    
    # lemmatize the word and remove stop words like a, an , the , is ,are ...
    news = [ps.lemmatize(word) for word in token if not word in stopwords]  
    
    # finaly join all the token with space
    cleanned_news = ' '.join(news) 
    
    # return cleanned data
    return cleanned_news 

In [37]:
dataset['text'] = dataset['text'].apply(lambda x : cleaning_data(x))

In [38]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [40]:
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))

In [41]:
dataset.shape

(44898, 2)

In [42]:
X = dataset.iloc[:35000,0]
y = dataset.iloc[:35000,1]

In [43]:
X.head()

13517    trump announce vp pick tomorrow rumor narrowed...
8847     washington reuters republican presidential can...
338      unless living rock since january know republic...
13817    texas imam agrees republican presidential cand...
22776    look around world throughout recent history fi...
Name: text, dtype: object

In [44]:

y.head()


13517    1
8847     0
338      1
13817    1
22776    1
Name: label, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
train_data , test_data , train_label , test_label = train_test_split(X , y , test_size = 0.2 ,random_state = 0)
vec_train_data = vectorizer.fit_transform(train_data)
vec_train_data = vec_train_data.toarray()

In [46]:
train_data.shape , test_data.shape

((28000,), (7000,))

In [48]:
vec_test_data = vectorizer.transform(test_data).toarray()


In [49]:
vec_train_data.shape , vec_test_data.shape

((28000, 50000), (7000, 50000))

In [50]:
train_label.value_counts() # balanced partition

1    14661
0    13339
Name: label, dtype: int64

In [52]:
test_label.value_counts() # balanced partition

1    3650
0    3350
Name: label, dtype: int64

In [54]:
training_data = pd.DataFrame(vec_train_data , columns=vectorizer.get_feature_names())
testing_data = pd.DataFrame(vec_test_data , columns= vectorizer.get_feature_names())

/Users/pranjalshukla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/pranjalshukla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
clf = MultinomialNB()
clf.fit(training_data, train_label)
y_pred  = clf.predict(testing_data)

In [60]:
pd.Series(y_pred).value_counts()

1    3703
0    3297
dtype: int64

In [61]:
test_label.value_counts()


1    3650
0    3350
Name: label, dtype: int64

In [62]:

print(classification_report(test_label , y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3350
           1       0.95      0.96      0.96      3650

    accuracy                           0.95      7000
   macro avg       0.96      0.95      0.95      7000
weighted avg       0.95      0.95      0.95      7000



In [63]:
y_pred_train = clf.predict(training_data)
print(classification_report(train_label , y_pred_train))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13339
           1       0.96      0.96      0.96     14661

    accuracy                           0.96     28000
   macro avg       0.96      0.96      0.96     28000
weighted avg       0.96      0.96      0.96     28000



In [69]:
accuracy_score(train_label , y_pred_train)

0.95925

In [70]:
accuracy_score(test_label , y_pred)

0.9547142857142857

In [71]:
news = cleaning_data(str("Imposters posing as army personnel on the social media have been called out by the Indian Army as false news and disinformation."))

In [72]:
single_prediction = clf.predict(vectorizer.transform([news]).toarray())
single_prediction


/Users/pranjalshukla/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


array([1])